In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} .cm-s-ipython span.cm-comment {color: #87CEFA; font-style: italic;}</style>"))

In [4]:
pd.options.display.max_columns = 1000

In [5]:
pd.options.display.max_rows = 1000

In [8]:
cust = pd.read_csv("CUST_DATA.csv", encoding="utf-16")
claim = pd.read_csv("CLAIM_DATA.csv", encoding="utf-16")

# 1. 여기부터 전처리 코드를 작성하세요.

## * AGE

In [9]:
cust.AGE = cust.AGE.map(lambda x: int(x//10))

In [10]:
# 전처리한 데이터를 임시 저장
cust.to_csv("CUST_DATA_1-1.csv", index=False, encoding="utf-8-sig")

## * CTPR (지역)
지역 관련 결측치 삭제시 열단위로 삭제하지 않도록 주의

In [11]:
# cust = cust.loc[~cust.CTPR.isnull()] # 결측치 행 삭제(약 600개?)
cust.CTPR = cust.CTPR.fillna('n')

In [12]:
##### 레이블 인코딩
# 지역 텍스트를 숫자로 인코딩 하기 위해, LabelEncoder 클래스를 사용
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(cust.CTPR)
ctpr = le.transform(cust.CTPR)

In [13]:
##### 원-핫 인코딩
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(ctpr.reshape(-1, 1))
ctpr_onehot = enc.transform(ctpr.reshape(-1,1))
ctpr_onehot.toarray()

# 이렇게 만들어진 ctpr_df 데이터프레임의 인덱스는 0부터 21779까지
ctpr_df = pd.DataFrame(ctpr_onehot.toarray(), columns=le.classes_)

# cust_df 데이터프레임은 중간의 행들이 삭제되어서 인덱스가 순서대로 되어있지 않음
# (아까 결측치 삭제해서)
# 따라서, 고객데이터와 원-핫 인코딩한 cust 데이터와 ctpr_df 데이터를 결합(concat)하기 전에,
# cust_df 데이터프레임의 인덱스를 초기화해야 함
cust.reset_index(drop=True, inplace=True)
cust = pd.concat([cust, ctpr_df], axis=1)

In [14]:
# 원본 데이터 열(CTPR)은 삭제
cust.drop("CTPR", axis=1, inplace=True)

In [15]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-2.csv", index=False, encoding="utf-8-sig")

## * OCCP_GRP_1 (직업 코드)

In [16]:
cust = pd.read_csv("CUST_DATA_1-2.csv", encoding="utf-8-sig")

In [17]:
# map 함수를 이용해서 직업코드 문자열에서 첫 문자만 추출
cust.OCCP_GRP_1 = cust.OCCP_GRP_1.map(lambda x: str(x)[0])

In [18]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-3.csv", index=False, encoding="utf-8-sig")

## * CUST_RGST(고객 등록 년월)

In [19]:
cust = pd.read_csv("CUST_DATA_1-3.csv", encoding="utf-8-sig")

In [20]:
from datetime import datetime
today_y = str(datetime.today().year)
today_m = str(datetime.today().month)
today = today_y + today_m

In [21]:
def calc_month(a, b):
    try:
        a = str(a)
        a_y = int(a[:4])
        a_m = int(a[4:6])

        b = str(b)
        b_y = int(b[:4])
        b_m = int(b[4:6])
    
        return (a_y - b_y) * 12 + (a_m - b_m) + 1
    except:
        return 0

In [22]:
cust["CUST_RGST_M"] = cust.CUST_RGST.map(lambda x: calc_month(today, x))

In [23]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-4.csv", index=False, encoding="utf-8-sig")

## 신용등급

In [24]:
len(cust.loc[cust.MAXCRDT.isnull()].index)

9476

In [25]:
cust.MINCRDT = cust.MINCRDT.fillna(6)
cust.MAXCRDT = cust.MAXCRDT.fillna(6)

In [26]:
cust_crdt = pd.DataFrame(data=(cust.MINCRDT + cust.MAXCRDT)/2, columns=["CRDT"])
cust = pd.concat([cust, cust_crdt], axis=1)

In [27]:
cust.drop(["MINCRDT", "MAXCRDT"], axis=1, inplace=True)

In [28]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-5.csv", index=False, encoding="utf-8-sig")

## 최대보험료 연월

In [29]:
max_ym = list()
for i in range(len(cust)):
    max_ym.append(calc_month(cust.MAX_PAYM_YM[i], cust.CUST_RGST[i]))

In [30]:
cust["MAX_PAYM_YM"] = max_ym

In [31]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-6.csv", index=False, encoding="utf-8-sig")

## 소득(개인소득/가구소득)

In [32]:
cust["HSHD_INCM_AVG"] = (cust.RCBASE_HSHD_INCM + cust.JPBASE_HSHD_INCM)/2

In [33]:
# 결측치 0으로 설정
cust.CUST_INCM = cust.CUST_INCM.fillna(0)
cust.HSHD_INCM_AVG = cust.HSHD_INCM_AVG.fillna(0)

In [34]:
cust.drop(["RCBASE_HSHD_INCM", "JPBASE_HSHD_INCM"], axis=1, inplace=True)

In [35]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_1-7.csv", index=False, encoding="utf-8-sig")

In [36]:
cust

,CUST_ID,DIVIDED_SET,SIU_CUST_YN,SEX,AGE,RESI_COST,RESI_TYPE_CODE,FP_CAREER,CUST_RGST,OCCP_GRP_1,OCCP_GRP_2,TOTALPREM,WEDD_YN,MATE_OCCP_GRP_1,MATE_OCCP_GRP_2,CHLD_CNT,LTBN_CHLD_AGE,MAX_PAYM_YM,MAX_PRM,CUST_INCM,n,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북,CUST_RGST_M,CRDT,HSHD_INCM_AVG
0,1,1,N,2,4,21111,20.0,N,199910.0,3,사무직,146980441.0,Y,3.사무직,2차산업 종사자,2.0,13.0,110,319718.0,4879.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,248,6.0,10715.5
1,2,1,N,1,5,40000,20.0,N,199910.0,3,사무직,94600109.0,Y,1.주부,주부,2.0,17.0,15,341341.0,6509.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,248,3.5,7826.0
2,3,1,N,1,6,0,NaN,N,199910.0,5,2차산업 종사자,18501269.0,N,NaN,NaN,0.0,0.0,44,131300.0,4180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,248,6.0,2090.0
3,4,1,N,2,6,12861,40.0,Y,199910.0,2,3차산업 종사자,317223657.0,N,NaN,NaN,0.0,0.0,132,1493184.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,248,50.5,5092.0
4,5,1,N,2,5,0,NaN,Y,199910.0,2,3차산업 종사자,10506072.0,Y,3.사무직,고위 공무원,3.0,19.0,106,166760.0,3894.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,248,8.0,4442.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22395,20048,2,NaN,1,3,5652,11.0,N,201103.0,2,자영업,NaN,N,NaN,NaN,0.0,0.0,0,NaN,3331.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111,7.5,3476.5
22396,21483,1,N,2,3,31944,20.0,N,200306.0,1,주부,NaN,N,NaN,NaN,0.0,0.0,0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,204,6.0,9154.0
22397,21940,1,N,2,2,45972,20.0,N,200501.0,8,학생,NaN,N,NaN,NaN,0.0,0.0,0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,185,6.0,11503.0
22398,20356,1,N,1,1,27777,20.0,N,200306.0,8,학생,NaN,N,NaN,NaN,0.0,0.0,0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,204,6.0,7787.0


## * 파생변수 추가
많이 추가할수록 좋다고 하셨음
### 고객별 평균 입원일

In [37]:
# 보험 청구 데이터에서 입원일은 VLID_HOSP_OTDA 열에 저장되어 있음
# 각 고객별로 청구데이터를 분석하여 고객별 평균 입원일을 집계하고, 이를 고객 데이터프레임에 추가해야 함
# 이에 관한 열 이름은 HOSP_DAY

In [38]:
# 고객의 아이디로 그룹핑 (DataFrameGroupBy 객체)
claim_g = claim.groupby(claim.CUST_ID)

# 그룹핑한 데이터의 평균을 계산하고 VLID_HOSP_OTDA 열만 따로 추출
claim_g_mean = claim_g.mean()
claim_hospday = claim_g_mean.loc[:, ["VLID_HOSP_OTDA"]]

# 행 인덱스를 초기화(행 인덱스는 행의 이름이 되도록)
# Result: 고객별 평균입원일을 저장한 데이터프레임
claim_hospday.reset_index(level=0, inplace=True)
claim_hospday.columns = ['CUST_ID', 'HOSP_DAY']

# 고객 데이터와 고객별 평균입원일을 저장한 데이터프레임을 병합
cust = pd.merge(cust, claim_hospday, how="inner")

In [39]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_2-1.csv", index=False, encoding="utf-8-sig")

### 사고원인, 청구코드

In [40]:
# 사용자 아이디, 사고원인, 청그코드 데이터 추출
claim_acci = claim.loc[:, ["CUST_ID", "ACCI_DVSN", "DMND_RESN_CODE"]]
claim_acci["value"] = 1

# 피벗 테이블 생성
cust_claim = claim_acci.pivot_table(index=["CUST_ID"], columns=["ACCI_DVSN", "DMND_RESN_CODE"],
                                         values=["value"], aggfunc='sum', fill_value=0)

cust_claim = cust_claim.reset_index(level=["CUST_ID"], col_level=1)
cust_claim.columns = cust_claim.columns.droplevel(level=0)

cust_claim.columns = ['_'.join([str(col) for col in cols]) for cols in cust_claim.columns]
cust_claim.rename(columns={'CUST_ID_':'CUST_ID'}, inplace=True)
cust = pd.merge(cust, cust_claim, how="left")

In [41]:
# 전처리한 데이터 임시 저장
cust.to_csv("CUST_DATA_2-2.csv", index=False, encoding="utf-8-sig")

In [42]:
cust.drop(["CUST_RGST", "OCCP_GRP_2", "WEDD_YN", "MATE_OCCP_GRP_1", "MATE_OCCP_GRP_2"], axis=1, inplace=True)

In [43]:
cust.drop(["FP_CAREER"], axis=1, inplace=True)

In [44]:
def to_int(x):
    try:
        return int(x)
    except:
        return 0

In [45]:
cust.OCCP_GRP_1 = cust.OCCP_GRP_1.map(to_int)

In [46]:
cust.drop(['n'], axis=1, inplace=True)

In [47]:
def yn_10(x):
    if x=='Y':
        return 1
    elif x=='N':
        return 0

In [48]:
cust.SIU_CUST_YN = cust.SIU_CUST_YN.map(lambda x: yn_10(x))

# PER_CHAR, PER_ITEM

In [49]:
# 고객의 아이디로 그룹핑 (DataFrameGroupBy 객체)
claim_g = claim.groupby(claim.CUST_ID)
# 그룹핑한 데이터의 평균을 계산
claim_g_mean = claim_g.mean()

# 환자별 부담 금액 평균
claim_pt_charge = claim_g_mean.loc[:,["PATT_CHRG_TOTA"]]

#환자별 부담 금액 결측치 0으로 처리
claim_pt_charge=claim_pt_charge.fillna(0) 

claim_pt_charge.reset_index(level=0, inplace=True)
claim_pt_charge.columns = ['CUST_ID', 'PER_CHAR']

cust = pd.merge(cust, claim_pt_charge, how="inner")

In [50]:
# 개인별 진료 과목 개수 평균
claim_pt_item = claim_g_mean.loc[:,["COUNT_TRMT_ITEM"]]

# 개인별 진료 과목 결측치 0으로 대체
claim_pt_item = claim_pt_item.fillna(0)

claim_pt_item.reset_index(level=0, inplace=True)
claim_pt_item.columns = ['CUST_ID', 'PER_ITEM']

cust = pd.merge(cust, claim_pt_item, how="inner")

In [51]:
claim_heed_hosp = claim.loc[:, ["CUST_ID", "HEED_HOSP_YN"]]
claim_heed_hosp = claim_heed_hosp.fillna(0)
claim_heed_hosp.HEED_HOSP_YN = claim_heed_hosp.HEED_HOSP_YN.map(lambda x: yn_10(x))
claim_heed_hosp_g = claim_heed_hosp.groupby(claim_heed_hosp.CUST_ID)
cust["HEED_HOSP_AVG"] = claim_heed_hosp.groupby("CUST_ID").sum()

In [52]:
# cust.drop(["HEED_HOSP_AVG"], axis=1, inplace=True)

In [53]:
claim_pay_ratio = claim.loc[:, ["CUST_ID", "NON_PAY_RATIO"]]
claim_pay_ratio.fillna(0, inplace=True)
claim_pay_ratio_g = claim_pay_ratio.groupby(claim_pay_ratio.CUST_ID)
cust["PAY_RATIO_AVG"] = claim_pay_ratio_g.mean()

In [54]:
# cust.drop(["PAY_RATIO_AVG"], axis=1, inplace=True)

In [55]:
claim_house_hosp = claim.loc[:, ["CUST_ID", "HOUSE_HOSP_DIST"]]
claim_house_hosp_g = claim_house_hosp.groupby(claim_house_hosp.CUST_ID)
cust["HOUSE_HOSP_DIST_AVG"] = claim_house_hosp_g.mean()

In [56]:
# cust.drop(["HOUSE_HOSP_DIST_AVG"], axis=1, inplace=True)

In [57]:
claim_fp = claim.loc[:, ["CUST_ID", "CHANG_FP_YN"]]
claim_fp.CHANG_FP_YN = claim_fp.CHANG_FP_YN.map(lambda x:yn_10(x))
claim_fp_g = claim_fp.groupby(claim_fp.CUST_ID)
cust["FP_DIFF_COUNT"] = claim_fp.groupby("CUST_ID").sum()

In [58]:
cust.drop(["FP_DIFF_COUNT"], axis=1, inplace=True)

In [59]:
claim_hosp = claim.loc[:, ["CUST_ID", "HOSP_CODE"]]
claim_hosp_d = claim_hosp.drop_duplicates()
claim_hosp_g = claim_hosp.groupby(claim_hosp.CUST_ID)
cust["HOST_DIFF_CNT"] = claim_hosp_g.count()

In [60]:
# cust.drop(["HOST_DIFF_CNT"], axis=1, inplace=True)

In [61]:
claim_dmnd_amt = claim.loc[:, ["CUST_ID", "DMND_AMT"]]
claim_dmnd_amt_g = claim_dmnd_amt.groupby(claim_hosp.CUST_ID)
cust["DMND_AMT_AVG"] = claim_dmnd_amt_g.mean()

ValueError: Wrong number of items passed 2, placement implies 1

In [62]:
# cust.drop(["DMND_AMT_AVG"], axis=1, inplace=True)

In [63]:
cust=cust.fillna(0)

# 여기부터는 수정하면 안됩니다.

# 전처리 완료된 파일을 저장하세요. 

In [64]:
cust.to_csv("CUST_DATA_전처리완료.csv", index=False, encoding="utf-8-sig")

In [65]:
cust.shape

(22400, 63)

### 전처리 완료된 데이터를 불러와서 훈련 데이터와 검증데이터로 나눔

In [66]:
cust = pd.read_csv("CUST_DATA_전처리완료.csv", encoding="utf-8-sig")

In [67]:
train = cust[cust['DIVIDED_SET']==1].reset_index(drop=True) # 사기자
test = cust[cust['DIVIDED_SET']==2].reset_index(drop=True) # 사기자가 아닌 사람

In [68]:
train.drop('DIVIDED_SET', axis=1, inplace=True)
test.drop('DIVIDED_SET', axis=1, inplace=True)

In [69]:
train_X = train.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)
train_y = train['SIU_CUST_YN']

In [70]:
test_X = test.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)

In [71]:
train['SIU_CUST_YN'].value_counts()

0.0    18801
1.0     1806
Name: SIU_CUST_YN, dtype: int64

In [72]:
train_X.shape, train_y.shape

((20607, 60), (20607,))

In [73]:
test_X.head()

,SEX,AGE,RESI_COST,RESI_TYPE_CODE,OCCP_GRP_1,TOTALPREM,CHLD_CNT,LTBN_CHLD_AGE,MAX_PAYM_YM,MAX_PRM,CUST_INCM,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북,CUST_RGST_M,CRDT,HSHD_INCM_AVG,HOSP_DAY,1_1,1_2,1_3,1_4,1_5,1_6,1_7,1_9,2_1,2_2,2_3,2_4,2_5,2_6,3_1,3_2,3_3,3_4,3_5,3_6,3_7,3_9,PER_CHAR,PER_ITEM,HEED_HOSP_AVG,PAY_RATIO_AVG,HOUSE_HOSP_DIST_AVG,HOST_DIFF_CNT
0,2,4,14027,20.0,1,53761263.0,2.0,19.0,20,442430.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227,6.0,5653.5,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.000000,6.774286,8.0
1,2,4,26250,20.0,2,162758082.0,1.0,20.0,172,2099765.0,4798.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,224,52.5,10213.5,4.875000,0,0,2,0,0,3,0,0,0,3,0,0,0,0,0,6,0,0,2,0,0,0,3351415.0,1.0,0.0,0.000000,8.480000,3.0
2,1,5,6707,12.0,3,13323745.0,0.0,0.0,11,197750.0,5126.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222,7.0,5266.0,0.333333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0.0,0.0,0.0,0.000000,11.630000,3.0
3,1,4,1603,13.0,6,5858640.0,3.0,13.0,15,49150.0,3660.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219,6.0,4280.0,8.000000,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.119836,0.515000,6.0
4,2,4,6250,30.0,5,91802397.0,2.0,22.0,92,799231.0,4279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,215,6.5,6808.0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.0,0.0,0.0,0.000000,11.455000,16.0


### Random Forest를 이용한 모델링

In [74]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_features=16, random_state=42)
rf_model.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=16,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [75]:
y_pred = rf_model.predict(test_X)

In [76]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [77]:
val_df = pd.read_csv("answer.csv") # 모평을 평가하기 위한 정답을 불러옴

In [78]:
val_df.head()

,CUST_ID,YN
0,37,0
1,51,0
2,60,0
3,65,0
4,73,0


In [79]:
pd.crosstab(val_df.YN, y_pred) # 교차분류표

col_0,0.0,1.0
YN,,
0,1564,71
1,151,7


In [80]:
rf_model.score(test_X, val_df.YN)

0.8761851645287229

### 10회 교차검증 후 f1 스코어의 평균을 출력합니다. 
### 출력한 값이 가장 큰 팀이 우승

In [81]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf_model, test_X, val_df.YN , cv=10, scoring='f1_macro')
scores.mean()

0.4826694372534638